In [5]:
import pandas as pd    
import io

In [6]:
df = pd.read_csv("../data/raw/dat-ab-usos-2025.csv")

In [8]:
# ============================================
# 2. Exploración de datos
# ============================================
print("\n--- Exploración Inicial ---")
print("\nPrimeras 5 filas:")
print(df.head())
print("\nInformación general y tipos de datos:")

buffer = io.StringIO()
df.info(buf=buffer)
s = buffer.getvalue()
print(s)
    
print("\nConteo de valores nulos por columna:")
print(df.isnull().sum())

print("\nEstadísticas descriptivas (para columnas numéricas):")
    
print(df.describe(include='number')) 


--- Exploración Inicial ---

Primeras 5 filas:
  DIA_TRANSPORTE                                     NOMBRE_EMPRESA  \
0     2025-01-01  MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...   
1     2025-01-01  MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...   
2     2025-01-01                                 EMPRESA BATAN S.A.   
3     2025-01-01                                 EMPRESA BATAN S.A.   
4     2025-01-01               COMPAÑIA  DE TRANSPORTE VECINAL S.A.   

              LINEA AMBA TIPO_TRANSPORTE JURISDICCION     PROVINCIA  \
0                 1   SI       COLECTIVO    MUNICIPAL  BUENOS AIRES   
1                2B   SI       COLECTIVO    MUNICIPAL  BUENOS AIRES   
2  BS_AS_LINEA 715M   NO       COLECTIVO    MUNICIPAL  BUENOS AIRES   
3  BS_AS_LINEA 720M   NO       COLECTIVO    MUNICIPAL  BUENOS AIRES   
4   BS_AS_LINEA_326   SI       COLECTIVO   PROVINCIAL  BUENOS AIRES   

            MUNICIPIO  CANTIDAD DATO_PRELIMINAR  
0            MERCEDES        23              NO 

In [9]:
# ============================================
# 3. Preprocesamiento y Agregación de Datos
# ============================================

print("\n--- Preprocesamiento y Agregación ---")
   

df['DIA_TRANSPORTE'] = pd.to_datetime(df['DIA_TRANSPORTE'])
print("\nColumna 'DIA_TRANSPORTE' convertida a datetime.")



print("Agrupando transacciones por día...")
df_diario = df.groupby('DIA_TRANSPORTE')['CANTIDAD'].sum().reset_index()

   
df_diario.set_index('DIA_TRANSPORTE', inplace=True)


df_diario.sort_index(inplace=True)

print("\nDataFrame agregado por día (df_diario) creado.")
print("Dimensiones de df_diario:", df_diario.shape)
print("\nPrimeras 5 filas de df_diario:")
print(df_diario.head())

# ============================================
# 4. Feature Engineering
# ============================================

print("\n--- Ingeniería de Características ---")

df_diario['dia_semana'] = df_diario.index.dayofweek # Monday=0, Sunday=6
df_diario['dia_mes'] = df_diario.index.day
   
df_diario['semana_anio'] = df_diario.index.isocalendar().week.astype(int)
df_diario['mes'] = df_diario.index.month
df_diario['trimestre'] = df_diario.index.quarter
df_diario['anio'] = df_diario.index.year

df_diario['es_finde'] = df_diario['dia_semana'].apply(lambda x: 1 if x >= 5 else 0)



print("\nNuevas características temporales agregadas a df_diario.")
print("\nPrimeras 5 filas de df_diario con nuevas características:")
print(df_diario.head())

print("\nValores nulos después de crear características:")
print(df_diario.isnull().sum())




--- Preprocesamiento y Agregación ---

Columna 'DIA_TRANSPORTE' convertida a datetime.
Agrupando transacciones por día...

DataFrame agregado por día (df_diario) creado.
Dimensiones de df_diario: (290, 1)

Primeras 5 filas de df_diario:
                CANTIDAD
DIA_TRANSPORTE          
2025-01-01       1718921
2025-01-02       9230763
2025-01-03      10286242
2025-01-04       7060658
2025-01-05       4085031

--- Ingeniería de Características ---

Nuevas características temporales agregadas a df_diario.

Primeras 5 filas de df_diario con nuevas características:
                CANTIDAD  dia_semana  dia_mes  semana_anio  mes  trimestre  \
DIA_TRANSPORTE                                                               
2025-01-01       1718921           2        1            1    1          1   
2025-01-02       9230763           3        2            1    1          1   
2025-01-03      10286242           4        3            1    1          1   
2025-01-04       7060658           5     